In [2]:
import os
import json
import pandas as pd

In [3]:
data_dir = '/mnt/nas2/GrimaRepo/datasets/20BN-SOMETHING-SOMETHING-V2'

In [8]:
prefix = 'something-something-v2-'
labels_json = json.load(open(os.path.join(data_dir, prefix + 'labels.json'), encoding='utf8'))
train_json = json.load(open(os.path.join(data_dir, prefix + 'train.json'), encoding='utf8'))
val_json = json.load(open(os.path.join(data_dir, prefix + 'validation.json'), encoding='utf8'))
test_json = json.load(open(os.path.join(data_dir, prefix + 'test.json'), encoding='utf8'))

In [9]:
def get_df(json_data):
    data = []
    for json_elem in json_data:
        video_id = json_elem['id']
        class_name = json_elem['template'].replace('[', '').replace(']', '')
        label = labels_json[class_name]
        data.append([video_id, label])
    df = pd.DataFrame(data, columns=['video_id', 'label'])
    
    return df

In [11]:
train_df = get_df(train_json)
val_df = get_df(val_json)
# test_df = get_df(test_json)

In [49]:
train_id_to_label = train_df.set_index('video_id')
val_id_to_label = val_df.set_index('video_id')

In [50]:
n_train = train_df.count()['label']
n_train_sample = int(.1 * n_train)

In [51]:
labels_by_size = train_df.groupby('label')['video_id'].count().sort_values(ascending=False)
sample_size = 0
sample_labels = []
for label, size in labels_by_size.iteritems():
    sample_size += size
    sample_labels.append(label)
    if sample_size >= n_train_sample:
        break

In [60]:
from tqdm import tqdm
def get_sample_json(original_json, original_id_to_label):
    sample_json = []
    for json_elem in tqdm(original_json):
        video_id = json_elem['id']
        label = original_id_to_label.loc[video_id]['label']
        if label in sample_labels:
            sample_json.append(json_elem)
    return sample_json

In [61]:
sample_train_json = get_sample_json(train_json, train_id_to_label)
sample_val_json = get_sample_json(val_json, val_id_to_label)

100%|██████████| 24777/24777 [00:03<00:00, 7078.89it/s]


In [68]:
json.dump(sample_train_json, open('train_10percent.json', 'w'))
json.dump(sample_val_json, open('val_10percent.json', 'w'))